In [1]:
"""
this file is just to make bilinear sampler easy to understand as the code is veing breaked.
"""

import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Conv2DTranspose
from tensorflow.keras import Model
import cv2
import numpy as np
import os

In [7]:
tf.__version__

'2.1.0'

In [4]:
B = 10 #baselength for two cameras
f = 2 #focal length
def left_to_right_transform(dp,Il_true,Ir_pred):
    #predicting the right image from left image
    print(dp.shape)
    for d in dp :
        Ir_pred = Il_true
    
def right_image_transform(dp,Ir_true,Il_pred):
    #predicting the left image from the right image 
    for x in range(dp,Ir_true.shape[1]):
        Il_pred[:,x] = Ir_true[:,x-dp]
    return Il_pred

def loss_object(y_pred):
    Dp = B*f/y_pred
    Il_pred = right_image_transform(Dp,Ir_true,Il_pred)

In [10]:
input_images = np.ones((2,200,100,3))*255#.reshape(1,100,100,3)
x_offset = np.ones((2,200,100,1))*1
#with tf.variable_scope('r'):
_num_batch    = np.shape(input_images)[0]
_height       = np.shape(input_images)[1]
_width        = np.shape(input_images)[2]
_num_channels = np.shape(input_images)[3]

_height_f = float(_height)
_width_f  = float(_width)

_wrap_mode = 'border'


#with tf.variable_scope('transform'):
# grid of (x_t, y_t, 1), eq (1) in ref [1]
x_t, y_t = np.meshgrid(np.linspace(0.0,   _width_f - 1.0,  _width),
                       np.linspace(0.0 , _height_f - 1.0 , _height))
#print('y_t_1',y_t)
#print('shape of x_t_1',x_t.shape)
#print('x_t_1',x_t)
#print('shape of y_t_1',y_t.shape)
#print(x_t)
x_t_flat = np.reshape(x_t, (1, -1))
y_t_flat = np.reshape(y_t, (1, -1))

#print('')
#print('')
#print('x_t_flat',x_t_flat)
#print('shape of x_t_flat',x_t_flat.shape)
#print('y_t_flat',y_t_flat)
#print('shape of y_t_flat',y_t_flat.shape)
#print(x_t)
#print(np.linspace(0.0,100-1,100))

x_t_flat = np.tile(x_t_flat, np.stack([_num_batch, 1]))
y_t_flat = np.tile(y_t_flat, np.stack([_num_batch, 1]))

#print(x_t_flat)

x_t_flat = np.reshape(x_t_flat, [-1])
y_t_flat = np.reshape(y_t_flat, [-1])

#print(x_t_flat.shape)
x_t_flat = x_t_flat+np.reshape(x_offset,[-1])*_width_f
#print('pixel_disparity',x_t_flat)
#print('y_t_2',y_t_flat)
#x_t_flat = x_t_flat + np.reshape(x_offset, [-1]) * _width_f

# handle both texture border types
_edge_size = 0
if _wrap_mode == 'border':
    _edge_size = 1
    im = np.pad(input_images, [[0, 0], [1, 1], [1, 1], [0, 0]], mode='constant')
    x = x_t_flat + _edge_size
    y = y_t_flat + _edge_size
elif _wrap_mode == 'edge':
    _edge_size = 0
#else:
    #return None

#x = .clip_by_value(x, 0.0,  _width_f - 1 + 2 * _edge_size)
x = np.clip(x, 0.0,  _width_f - 1 + 2 * _edge_size)

x0_f = np.floor(x)
y0_f = np.floor(y)

x1_f = x0_f + 1

#print('x0_f',x0_f)
#print('x1_f',x1_f)


x0 = x0_f.astype(np.int32)
y0 = y0_f.astype(np.int32)
x1 = tf.cast(tf.minimum(x1_f,  _width_f - 1 + 2 * _edge_size), tf.int32)
#print('y0',y0)

dim2 = (_width + 2 * _edge_size)
dim1 = (_width + 2 * _edge_size) * (_height + 2 * _edge_size)
#base = _repeat(
x_repeat = np.arange(_num_batch) * dim1
n_repeats =  _height * _width
rep = np.tile(np.expand_dims(x_repeat, 1), [1, n_repeats])
base = np.reshape(rep, [-1])
#print('base',base)
#print('rep',rep)
#print('dim2',dim2)
base_y0 = base + y0 * dim2#why
#print('base_y0',base_y0)
idx_l = base_y0 + x0
print('idx_l',idx_l)
idx_r = base_y0 + x1
print('idx_r',idx_r)

im_flat = np.reshape(im, np.stack([-1, _num_channels]))

pix_l = tf.gather(im_flat, idx_l)
pix_r = tf.gather(im_flat, idx_r)
print('pix_l',pix_l)
print('pix_r',pix_r)

weight_l = np.expand_dims(x1_f - x, 1)
weight_r = np.expand_dims(x - x0_f, 1)

input_transformed = weight_l * pix_l + weight_r * pix_r


#
#input_transformed = _interpolate(input_images, x_t_flat, y_t_flat)

output = tf.reshape(input_transformed, tf.stack([_num_batch, _height, _width, _num_channels]))

idx_l [  203   203   203 ... 41105 41105 41105]
idx_r tf.Tensor([  203   203   203 ... 41105 41105 41105], shape=(40000,), dtype=int32)
pix_l tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(40000, 3), dtype=float64)
pix_r tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(40000, 3), dtype=float64)
